### Melon 100 Chart 
* 100곡의 노래의 제목, Song ID추출
* Song ID로 노래의 상세정보를 100 번 요청
* 100곡의 정보를 json에 저장
* json file을 읽어서 Pandas DataFrame에 저장해서 표데이터로 만들기
* 표데이터를 DB에 저장

In [2]:
import requests
from bs4 import BeautifulSoup
import re

req_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
}
req_header

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}